<a href="https://colab.research.google.com/github/leox1v/Optimization_tutorial/blob/master/Optimization_DL2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Imports
%tensorflow_version 2.x
import tensorflow as tf

# Load MNIST

In [5]:
# get the dataset
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Let's press the data between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

11493376/11490434 [==============================] - 0s 0us/step


## Model

In [0]:
def get_model():
  return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')])

# SGD
$$ 
    \theta^+ = \theta - \eta \nabla L(x_{i:i+n}; \theta)
$$

In [0]:
class MySGD(tf.keras.optimizers.Optimizer):
    def __init__(self, learning_rate=0.01):
        super(MySGD, self).__init__(name='my_sgd')
        self.learning_rate = learning_rate
        
    def apply_gradients(self, grads_and_vars): 
      # use variable.assign_sub(value) to modify the variable value 
        for gradient, variable in grads_and_vars:
          variable.assign_sub(self.learning_rate*gradient)

    def get_config(self):
      config = {
          'lr': self.learning_rate
      }
      base_config = super(SGD, self).get_config()
      return dict(list(base_config.items()) + list(config.items()))

# Adagrad 
$$
    g_{t,i} = \nabla_\theta L(x_t; \theta_{t,i})\\
    G_{t,i} = \sum_{\tau=0}^t g_{\tau, i}^2\\
    \theta^+ = \theta - \frac{\eta}{\sqrt{G_{t,i} + \epsilon}} g_{t,i}
$$

In [0]:
class MyAdagrad(tf.keras.optimizers.Optimizer):
    def __init__(self, learning_rate=0.01, epsilon=0.1):
        super(MyAdagrad, self).__init__(name='MyAdagrad')
        self.learning_rate = learning_rate
        self.epsilon = epsilon
        self.g = None
        
    def apply_gradients(self, grads_and_vars):
        if self.g is None:
            grads_and_vars = list(grads_and_vars)
            [print('Weights {}: gradient of shape {}'.format(i, grad.shape)) for i, (grad, variable) in enumerate(grads_and_vars)]
            self.g = [tf.ones_like(gradient_layer, dtype=tf.float32) * self.epsilon for (gradient_layer, _) in grads_and_vars]
            
        for i, (gradient, variable) in enumerate(grads_and_vars):
            self.g[i] += tf.square(gradient)
            variable.assign_sub(self.learning_rate/tf.sqrt(self.g[i])*gradient)

    def get_config(self):
      config = {
          'lr': self.learning_rate,
          'epsilon': self.epsilon
      }
      base_config = super(SGD, self).get_config()
      return dict(list(base_config.items()) + list(config.items()))

# Adam
https://arxiv.org/pdf/1412.6980.pdf see improved algorithm at the end of section 2.
$$
    m_t = \beta_1 m_{t-1} + (1-\beta_1) \nabla_\theta L(x_t; \theta_t)\\
    v_t = \beta_2 v_{t-1} + (1-\beta_2) \left(\nabla_\theta L(x_t; \theta_t)\right) ^2\\
    \eta_t = \eta * \frac{\sqrt{1-\beta_2^t}}{1-\beta_1^t}
$$
$$
    \theta_{t+1} = \theta_t - \frac{\eta_t}{\sqrt{v_t} + \epsilon}m_t
$$

In [0]:
class MyAdam(tf.keras.optimizers.Optimizer):
    def __init__(self, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-07):
        self.learning_rate = learning_rate
        self.beta1 = beta1
        self.beta2 = beta2
        self.epsilon = epsilon
        self.m = None
        self.v = None
        self.iteration = 0
        
    def apply_gradients(self, grads_and_vars):
        self.iteration += 1
        if self.m is None:
            grads_and_vars = list(grads_and_vars)
            self.m = [tf.zeros_like(gradient_layer) for (gradient_layer, _) in grads_and_vars]
            self.v = [tf.zeros_like(gradient_layer) for (gradient_layer, _) in grads_and_vars]

        learning_rate = self.learning_rate * tf.sqrt(1-self.beta2**self.iteration)/(1-self.beta1**self.iteration)
        
        for i, (gradient, variable) in enumerate(grads_and_vars):
            self.m[i] = self.beta1 * self.m[i] + (1 - self.beta1) * gradient
            self.v[i] = self.beta2 * self.v[i] + (1 - self.beta2) * tf.square(gradient)
            update_step = learning_rate / (tf.sqrt(self.v[i]) + self.epsilon) * self.m[i]
            variable.assign_sub(update_step)

    def get_config(self):
      config = {
          'lr': self.learning_rate,
          'epsilon': self.epsilon,
          'beta1': self.beta1,
          'beta2': self.beta2
      }
      base_config = super(SGD, self).get_config()
      return dict(list(base_config.items()) + list(config.items()))

# Train the model

In [17]:
# SGD
model = get_model()
model.compile(optimizer=MySGD(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=3)

Train on 60000 samples
Epoch 1/3
60000/60000 [==============================] - 6s 92us/sample - loss: 0.6398 - accuracy: 0.8379
Epoch 2/3
60000/60000 [==============================] - 5s 87us/sample - loss: 0.3383 - accuracy: 0.9060
Epoch 3/3
60000/60000 [==============================] - 5s 88us/sample - loss: 0.2906 - accuracy: 0.9175


In [15]:
# Adagrad
model = get_model()
model.compile(optimizer=MyAdagrad(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=3)

Train on 60000 samples
Epoch 1/3
Weights 0: gradient of shape (784, 128)
Weights 1: gradient of shape (128,)
Weights 2: gradient of shape (128, 10)
Weights 3: gradient of shape (10,)
60000/60000 [==============================] - 6s 94us/sample - loss: 0.4357 - accuracy: 0.8818
Epoch 2/3
60000/60000 [==============================] - 6s 92us/sample - loss: 0.2437 - accuracy: 0.9313
Epoch 3/3
60000/60000 [==============================] - 5s 91us/sample - loss: 0.1917 - accuracy: 0.9462


In [16]:
# Adam
model = get_model()
model.compile(optimizer=MyAdam(),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=3)

Train on 60000 samples
Epoch 1/3
60000/60000 [==============================] - 6s 96us/sample - loss: 0.3775 - accuracy: 0.9061
Epoch 2/3
60000/60000 [==============================] - 6s 95us/sample - loss: 0.3134 - accuracy: 0.9428
Epoch 3/3
60000/60000 [==============================] - 6s 94us/sample - loss: 0.2757 - accuracy: 0.9534
